### Install Dependencies

In [2]:
# Might need to install tesseract into your system separately
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13631 sha256=20591e7fc9354fa2eb9b7269d58c6bb6f3d4382ba7ae7e95e635e3a19248d988
  Stored in directory: /Users/xavierpuspus/Library/Caches/pip/wheels/be/c9/54/c3a5693600be95ece493b981ae104c94e1a2c4d3e434438810
Successfully built pytesseract


### Write Text Detection

In [14]:
import pytesseract
from PIL import Image
class TextDetectionFromImage:
  """
  Simple python class to detet images from object
  """
  def __init__(self):
    pass

  def detect_text(self, image):
    im = Image.open(image)
    text = pytesseract.image_to_string(im)
    clean_text = text.replace(" \n\x0c", "").replace("\n", "")
    text_exists = clean_text != ''
    
    return text_exists, clean_text

text_detector = TextDetectionFromImage()

In [15]:
text_detector.detect_text('images/plate_number.jpg')

(True, ' eo  _sfg esats dicbialias | «')

### Write service file

In [28]:
pytesseract

<module 'pytesseract' from '/opt/anaconda3/lib/python3.8/site-packages/pytesseract/__init__.py'>

In [29]:
%%writefile text_detection_service.py

from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import PickleArtifact
from bentoml.adapters import ImageInput

@artifacts([PickleArtifact('model')])
@env(pip_dependencies=['pytesseract-0.3.6'], 
     conda_channels=["conda-forge"], 
     conda_dependencies=["ruamel.yaml"])
class TextDetectionService(BentoService):
    
    @api(input=ImageInput())
    def predict(self, image):
        result = self.artifacts.model.detect_text(image)
        return result

Overwriting text_detection_service.py


### Package Model and Serve

In [30]:
# 1) import the custom BentoService defined above
from text_detection_service import TextDetectionService

# 2) `pack` it with required artifacts
bento_service = TextDetectionService()
bento_service.pack("model", text_detector)

# 3) save your BentoService to file archive
saved_path = bento_service.save()

[2020-09-09 16:14:51,836] INFO - BentoService bundle 'TextDetectionService:20200909161450_4B2FEE' saved to: /Users/xavierpuspus/bentoml/repository/TextDetectionService/20200909161450_4B2FEE


### Docker Build

!cd {saved_path} && docker build -t text-detector .

In [31]:
!pipreqs .

INFO: Successfully saved requirements file in ./requirements.txt
